In [1]:
from OurTrainingTools import *

In [2]:
class OurModel(nn.Module):
### Defines the  model with parametrized discriminant. Only quadratic dependence on a single parameter is implemented.
### Input is the architecture (list of integers, the last one being equal to 1) and the activation type ('ReLU' or 'Sigmoid')
    def __init__(self, AR = [1, 3, 3, 1] , AF = 'ReLU' ):               
        super(OurModel, self).__init__() 
        ValidActivationFunctions = {'ReLU': torch.relu, 'Sigmoid': torch.sigmoid}
        try:
            self.ActivationFunction = ValidActivationFunctions[AF]
        except KeyError:
            print('The activation function specified is not valid. Allowed activations are %s.'
                 %str(list(ValidActivationFunctions.keys())))
            print('Will use ReLU.')
            self.ActivationFunction = torch.relu            
        if type(AR) == list:
            if( ( all(isinstance(n, int) for n in AR)) and ( AR[-1] == 1) ):
                self.Architecture = AR
            else:
                print('Architecture should be a list of integers, the last one should be 1.')
                raise ValueError             
        else:
            print('Architecture should be a list !')
            raise ValueError

### Define Layers
        self.LinearLayerList1  = nn.ModuleList([nn.Linear(self.Architecture[i], 
            self.Architecture[i+1]) for i in range(len(self.Architecture)-2)])
        self.OutputLayer1 = nn.Linear(self.Architecture[-2], 1)       
        self.LinearLayerList2 = nn.ModuleList([nn.Linear(self.Architecture[i], 
            self.Architecture[i+1]) for i in range(len(self.Architecture)-2)])
        self.OutputLayer2 = nn.Linear(self.Architecture[-2], 1)
        
        #self.Optimiser = torch.optim.Adam(self.parameters(), self.InitialLearningRate)
        #self.Criterion = WeightedMSELoss()

    def Forward(self, Data, Parameters):
### Forward Function. Performs Preprocessing, returns F = rho/(1+rho) in [0,1], where rho is quadratically parametrized.
        # Checking that data has the right input dimension
        InputDimension = self.Architecture[0]
        if Data.size(1) != InputDimension:
            print('Dimensions of the data and the network input mismatch: data: %d, model: %d'
                  %(Data.size(1), InputDimension))
            raise ValueError

        # Checking that preprocess has been initialised
        if not hasattr(self, 'Shift'):
            print('Please initialize preprocess parameters!')
            raise ValueError
        with torch.no_grad(): 
            Data, Parameters = self.Preprocess(Data, Parameters)  
        
        x1 = x2 = Data
        
        for i, Layer in enumerate(self.LinearLayerList1):
            x1 = self.ActivationFunction(Layer(x1)) 
        x1 = self.OutputLayer1(x1).squeeze()
        
        for i, Layer in enumerate(self.LinearLayerList2):
            x2 = self.ActivationFunction(Layer(x2))
        x2 = self.OutputLayer2(x2).squeeze()
        
        rho = (1 + torch.mul(x1, Parameters))**2 + (torch.mul(x2, Parameters))**2  
        return (rho.div(1.+rho)).view(-1, 1)
    
    def GetL1Bound(self, L1perUnit):
### Max L1 norm of weights at each layer. What about bias? Excluding the first layer?
        self.L1perUnit = L1perUnit
        L1MaxList = []
        for m in self.children():
            if isinstance(m, nn.Linear):
                L1MaxList.append(m.weight.size(0)*m.weight.size(1) \
                                  *self.L1perUnit)
            else:
                for mm in m:
                    L1MaxList.append(mm.weight.size(0)*mm.weight.size(1)\
                                      *self.L1perUnit)
        self.L1MaxList = L1MaxList
        print('L1MaxList created.')
    
    def ClipL1Norm(self):
### Clip the weights      
        def ClipL1NormLayer(DesignatedL1Max, Layer):
            L1 = Layer.weight.abs().sum()
            Layer.weight.masked_scatter_(L1 > DesignatedL1Max, 
                                        Layer.weight*(DesignatedL1Max/L1))
        
        Counter = 0
        for m in self.children():
            if isinstance(m, nn.Linear):
                Counter += 1
                with torch.no_grad():
                    DesignatedL1Max = self.L1MaxList[counter-1]
                    if Counter != 1: ClipL1NormLayer(DesignatedL1Max, m)
                    ### this avoids clipping the first layer
            else:
                for mm in m:
                    Counter +=1
                    with torch.no_grad():
                        DesignatedL1Max = self.L1MaxList[counter-1]
                        if Counter != 1: ClipL1NormLayer(DesignatedL1Max, mm)
                        ### this avoids clipping the first layer
        return 
    
    def DistributionRatio(self, points):
### This is rho. I.e., after training, the estimator of the distribution ratio.
        with torch.no_grad():
            F = self(points)
        return F/(1-F)

    def InitPreprocess(self, Data, Parameters):
### This can be run only ONCE to initialize the preprocess (shift and scaling) parameters
### Takes as input the training Data and the training Parameters as Torch tensors.
        if not hasattr(self, 'Scaling'):
            print('Initializing Preprocesses Variables')
            self.Scaling = Data.std(0)
            self.Shift = Data.mean(0)
            self.ParameterScaling = Parameters.std(0)  
        else: print('Preprocess can be initialized only once. Parameters unchanged.')
            
    def Preprocess(self, Data, Parameters):
### Returns scaled/shifted data and parameters
### Takes as input Data and Parameters as Torch tensors.
        if  not hasattr(self, 'Scaling'): print('Preprocess parameters are not initialized.')
        Data = (Data - self.Shift)/self.Scaling
        Parameters = Parameters/self.ParameterScaling
        return Data, Parameters
    
    def Save(self, Name, Folder):
### Saves the model in Folder/Name
        FileName = Folder + Name + '.pth'
        torch.save({'StateDict': self.state_dict(), 
                   'Scaling': self.Scaling,
                   'Shift': self.Shift,
                   'ParameterScaling': self.ParameterScaling}, 
                   FileName)
        print('Model successfully saved.')
        print('Path: %s'%str(FileName))
    
    def Load(self, Name, Folder):
### Loads the model from Folder/Name
        FileName = Folder + Name + '.pth'
        try:
            IncompatibleKeys = self.load_state_dict(torch.load(FileName)['StateDict'])
        except KeyError:
            print('No state dictionary saved. Loading model failed.')
            return 
        
        if list(IncompatibleKeys)[0]:
            print('Missing Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        if list(IncompatibleKeys)[1]:
            print('Unexpected Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        self.Scaling = torch.load(ModelPath + Name + '.pth')['Scaling']
        self.Shift = torch.load(ModelPath + Name + '.pth')['Shift']
        self.ParameterScaling = torch.load(ModelPath + Name + '.pth')['ParameterScaling']
        
        print('Model successfully loaded.')
        print('Path: %s'%str(FileName))
        
    def Report(self): ### is it possibe to check if the model is in double?
        print('\nModel Report:')
        print('Preprocess Initialized: ' + str(hasattr(self, 'Shift')))
        print('Architecture: ' + str(self.Architecture))
        print('Loss Function: ' + 'Quadratic')
        print('Activation: ' + str(self.ActivationFunction))
        
    def cuda(self):
        nn.Module.cuda(self)
        self.Shift = self.Shift.cuda()
        self.Scaling = self.Scaling.cuda()
        self.ParameterScaling = self.ParameterScaling.cuda()
        
    def cpu(self):
        self.Shift = self.Shift.cpu()
        self.Scaling = self.Scaling.cpu()
        self.ParameterScaling = self.ParameterScaling.cpu()
        return nn.Module.cpu(self)

In [3]:
#del TD
TD = OurTrainingData(['/data3/Training/TrainingData/SMData/ChPsm2.h5'], 
                     ['/data3/Training/TrainingData/GWData/ChPgw2.h5', '/data3/Training/TrainingData/GWData/ChPgwm2.h5',
                      '/data3/Training/TrainingData/GWData/ChPgw5.h5'],
                     parameters = ['GW[TeV**-2]'], process = 'W+Z', 
                     verbose=False)# , BSMNLimits = [ 300000, 300000, 300000 ] )

In [4]:
#TD = OurTrainingData(['/data3/Training/TrainingData/GphiData/ChPsm.h5'], 
 #                    ['/data3/Training/TrainingData/GphiData/ChPgphi50.h5','/data3/Training/TrainingData/GphiData/ChPgphim50.h5'],
  #                   parameters = ['Gphi[TeV**-2]'], process = 'W+Z', 
   #                  verbose=False)# , BSMNLimits = [ 300000, 300000, 300000 ] )

In [23]:
import copy
def OurCudaTensor(input):
    output = copy.deepcopy(input)
    output = output.cuda()
    return output

class OurTrainer(nn.Module):
### Contains all parameters for training: Loss Function, Optimiser, NumberOfEpochs, InitialLearningRate, SaveAfterEpoch 
    def __init__(self, LearningRate = 1e-3, LossFunction = 'Quadratic', Optimiser = 'Adam', NumEpochs = 100):
        super(OurTrainer, self).__init__() 
        self.NumberOfEpochs = NumEpochs
        self.InitialLearningRate = LearningRate
        ValidCriteria = {'Quadratic': WeightedMSELoss()}
        try:
            self.Criterion = ValidCriteria[LossFunction]
        except KeyError:
            print('The loss function specified is not valid. Allowed losses are %s.'
                 %str(list(ValidCriteria)))
            print('Will use Quadratic Loss.') 
        ValidOptimizers = {'Adam': torch.optim.Adam}
        try:
            self.Optimiser =  ValidOptimizers[Optimiser]
        except KeyError:
            print('The specified optimiser is not valid. Allowed optimisers are %s.'
                 %str(list(ValidOptimisers)))
            print('Will use Adam.')          
    
    def EstimateRequiredGPUMemory(self, model, Data, Parameters):
        if next(model.parameters()).is_cuda:
            print('Model is on cuda. No estimate possible anymore.')
            return None
        else:
            before = torch.cuda.memory_allocated()
            print(before)
            ### Always make deep copy of objects before sending them to cuda. Delete when done
            ModelCuda = copy.deepcopy(model)
            ModelCuda.cuda()
            DataCuda = OurCudaTensor(Data[:10000])
            ParametersCuda = OurCudaTensor(Parameters[:10000])
            print(torch.cuda.memory_allocated())
            MF = ModelCuda.Forward(DataCuda, ParametersCuda)
            after = torch.cuda.memory_allocated()
            print(after)
            del ModelCuda, DataCuda, ParametersCuda, MF
            torch.cuda.empty_cache()        
            estimate = float(Data.size()[0])/1e4*float(after-before)*1e-9
            print(str(estimate) + ' GB')
            return estimate
        
    def Train(self, model, Data, Parameters, Labels, Weights, L1perUnit=None, UseGPU=True, Name="", Folder=os.getcwd()):
        
        tempmodel = copy.deepcopy(model)
        tempmodel.cuda()
        tempData = OurCudaTensor(Data)
        tempParameters = OurCudaTensor(Parameters)
        tempLabels = OurCudaTensor(Labels)
        tempWeights = OurCudaTensor(Weights)
        
        self.Optimiser = self.Optimiser(tempmodel.parameters(), self.InitialLearningRate)
        mini_batch_size = 100
        for e in range(self.NumberOfEpochs):
            print("epoch")
            self.Optimiser.zero_grad()
            for b in range(0, Data.size(0), mini_batch_size):
                torch.cuda.empty_cache()
                output          = tempmodel.Forward(tempData[b:b+mini_batch_size], tempParameters[b:b+mini_batch_size])
                loss            = self.Criterion(output, tempLabels[b:b+mini_batch_size].reshape(-1,1), 
                                                 tempWeights[b:b+mini_batch_size].reshape(-1, 1))               
                loss.backward()
            self.Optimiser.step()
        return tempmodel.cpu()

In [20]:
#del OT
torch.cuda.empty_cache()
torch.cuda.memory_allocated()

0

In [18]:
TD.ParVal.size(0)

2309217

In [ ]:
MD = OurModel(AR=[11,32,32,32,1])
MD.double()
MD.InitPreprocess(TD.Data, TD.ParVal)
print(MD.Forward(TD.Data[:1],TD.ParVal[:1]))
OT = OurTrainer(NumEpochs = 100)
MD = OT.Train(MD,Data = TD.Data, Parameters = TD.ParVal, Labels=TD.Labels, Weights=TD.Weights)
print(MD.Forward(TD.Data[:1],TD.ParVal[:1]))
print(torch.cuda.memory_allocated())
del OT
torch.cuda.empty_cache()
print(MD.Forward(TD.Data[:1],TD.ParVal[:1]))
torch.cuda.memory_allocated()

Initializing Preprocesses Variables
tensor([[0.4892]], dtype=torch.float64, grad_fn=<ViewBackward>)
epoch
epoch
epoch
epoch


In [23]:
#del MD
MD = OurModel(AR=[11,32,32,32,1])
print(MD.Architecture)
MD.double()
MD.InitPreprocess(TD.Data, TD.ParVal)
OT = OurTrainer()
OT.EstimateRequiredGPUMemory(MD, TD.Data, TD.ParVal)
del MD

[11, 32, 32, 32, 1]
Initializing Preprocesses Variables
0
1006080
33580032
7.754358075494401 GB


In [37]:
#del MD
MD = OurModel(AR=[11,200,200,1])
print(MD.Architecture)
MD.double()
MD.InitPreprocess(TD.Data, TD.ParVal)
OT = OurTrainer(MD)
OT.EstimateRequiredMemory(TD.Data, TD.ParVal)
del MD

[11, 200, 200, 1]
Initializing Preprocesses Variables
0


RuntimeError: CUDA out of memory. Tried to allocate 764.00 MiB (GPU 0; 7.92 GiB total capacity; 5.31 GiB already allocated; 764.25 MiB free; 5.33 GiB reserved in total by PyTorch)

In [8]:
MD = OurModel(AR=[11,200,200,200,1])
print(MD.Architecture)
MD.double()
MD.InitPreprocess(TD.Data, TD.ParVal)
MD.cuda()
MD.Forward(TD.Data.cuda(), TD.ParVal.cuda())

[11, 200, 200, 200, 1]
Initializing Preprocesses Variables


RuntimeError: CUDA out of memory. Tried to allocate 3.44 GiB (GPU 0; 7.92 GiB total capacity; 3.86 GiB already allocated; 2.04 GiB free; 4.03 GiB reserved in total by PyTorch)

In [18]:
#del OT
#torch.cuda.empty_cache() 
torch.cuda.memory_allocated()

0

In [48]:
next(OT.Model.parameters()).is_cuda

False

In [ ]:
Data.element_size()*Data.nelement()

In [26]:
TD.Data[:1000].size()[0]

1000

In [6]:
TD.Data.cuda()
torch.cuda.memory_allocated()

0

In [30]:
dd = TD.Data[:10000].cuda()
print(torch.cuda.memory_allocated())
del dd

dd = TD.Data[:100000].cuda()
print(torch.cuda.memory_allocated())
del dd

885760
8805888


In [10]:
dd.is_cuda

True

In [22]:
#del MD
#del OT
#torch.cuda.empty_cache()
#LD=MD
#LD.cuda()
torch.cuda.memory_allocated()

0

In [25]:
next(MD.parameters()).is_cuda

NameError: name 'MD' is not defined

In [20]:
TR = OurTrainer(MD)
print(MD.ParameterScaling)
TR.EstimateRequiredMemory(TD.Data, TD.ParVal)

tensor(0.0288, dtype=torch.float64)
0
5632
-5632
1.3113126400000001GB


In [8]:
TD.Data.size()[0]

1486086

In [6]:
TD = OurTrainingData(['/data3/Training/TrainingData/GWData/ChPsm.h5', '/data3/MadGraph/testSM1.h5'], 
                     ['/data3/Training/TrainingData/GWData/ChPgw2.h5', '/data3/Training/TrainingData/GWData/ChPgwm2.h5',
                      '/data3/Training/TrainingData/GWData/ChPgw5.h5'],
                    ['GW[TeV**-2]'], verbose=False, SMNLimits = [ 10000, 1000 ], BSMNLimits = [ 10000, 1000, 2000 ] )
print(TD.SMData.size())
print(TD.SMData.dtype)
print([w.size() for w in TD.BSMWeightsList])
print([w.dtype for w in TD.BSMWeightsList])
TD.Report()

torch.Size([11000, 11])
torch.float64
[torch.Size([10000]), torch.Size([1000]), torch.Size([2000])]
[torch.float64, torch.float64, torch.float64]

Loaded SM Files:
  ['GW[TeV**-2]']    #Data    XS[pb](avg.w)
-----------------  -------  ---------------
                0   566980      0.000322493
                0    19108      0.000325419

Loaded BSM Files:
  ['GW[TeV**-2]']    #Data    XS[pb](avg.w)
-----------------  -------  ---------------
             0.02   571169      0.000324063
            -0.02   573538      0.000329051
             0.05   597532      0.000340505

Paired BSM/SM Datasets:

  ['GW[TeV**-2]']    #Ev.BSM    #Ev.SM    sum.w SM\/XSSM
-----------------  ---------  --------  ----------------
             0.02      10000      8461          9979.43
            -0.02       1000       846           974.481
             0.05       2000      1692          2045.99



# Init test2

In [ ]:
import h5py, torch, time, datetime, os
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn.modules import Module
from tabulate import tabulate

In [ ]:
class DataFile():
### Reads sample file Info (string), Parameters (list), Values (torch array), Data (torch array) and Weights (torch array)
### FilePath is the path of the input file
### Computes cross-section XS (average weight) and total number of data ND in file
### Checks that files are in correct format (correct Keys)
### and that the length of Parameters and Data equals the one of Values and Weights respectively
    def __init__(self, FilePath, verbose=True):
        print('\nReading file ...' + FilePath)
        file = h5py.File(FilePath, 'r')
        if list(file.keys()) == ['Data', 'Info', 'Parameters', 'Values', 'Weights']:
            if( (len(file['Parameters'][()]) == len(file['Values'][()])) and (len(file['Data'][()]) == len(file['Weights'][()])) ):
                if verbose: print('##### File Info:\n' + file['Info'][()][0] + '\n#####')
                self.FilePath = FilePath
                self.Info = file['Info'][()][0]
                self.Parameters = file['Parameters'][()]
                #print(file['Values'][()].dtype)
                #print('%.15f' % (file['Data'][()][0][0]))
                self.Values = torch.DoubleTensor(file['Values'][()])
                self.Data = torch.DoubleTensor(file['Data'][()])
                self.Weights = torch.DoubleTensor(file['Weights'][()])
                self.XS = self.Weights.mean()
                self.ND = len(self.Weights)
            else: print('--> File not valid:\nunequal lenght of Values and Parameters or of Data and Weights')
        else:
            print('--> File format not valid:\nKeys: ' + print(list(file.keys())) + 
                  'should be ' + print(['Data', 'Info', 'Parameters', 'Values', 'Weights']))
            
#test = []
#test.append(DataFile('/data3/Training/TrainingData/GWData/ChPsm.h5'))
#test.append(DataFile('/data3/Training/TrainingData/GWData/ChPgw10.h5'))
#test.append(DataFile('/data3/MadGraph/testSM1.h5'))

test = [ DataFile(path) for path in ['/data3/Training/TrainingData/GWData/ChPsm.h5', '/data3/MadGraph/testSM1.h5', '/data3/MadGraph/testSM1.h5'] ]

In [ ]:
print(test[2].Values.tolist())
print(test[2].FilePath)
print(test[2].Values.dtype)
print(test[2].Weights.dtype)
print(test[2].Data.dtype)
print(test[2].XS.dtype)
print(type(test[2].ND))
print('%.15f' % test[2].Weights[0])
print('%.15f' % test[2].Data[0][2])
print('%.15f' % test[2].XS)
print('%.15f' % test[0].XS)
print((1.0-test[2].XS/test[0].XS).dtype)
print('%.15f' % (1.0-test[2].XS/test[0].XS))

In [ ]:
print(test[0].Values)
print(test[1].Values)
print(test[1].XS)
print(test[1].ND)
print(type(test[0].ND))
len(test[1].Weights)

In [ ]:
test = SampleFile('/data3/Training/TrainingData/GWData/ChMsm.h5')
print(test.Values.dtype)
print(test.Data.dtype)
print(test.Weights.dtype)

In [ ]:
a=torch.DoubleTensor([2.,3.4])
print(a)
a = a*4./8.
print(a)

In [ ]:
class OurTrainingData():
    def __init__(self, SMfilepathlist, BSMfilepathlist, parameters, SMNLimits="NA", BSMNLimits="NA", verbose=True): 
        self.Parameters = parameters
        print('Loading Data Files with Parameters: ' + str(self.Parameters) ) 
        if len(self.Parameters)!= 1: print('Only 1D Implemented in Training !')   
  
        
####### Load BSM data (stored in self.BSMDataFiles)
        if type(BSMfilepathlist) == list:
            if all(isinstance(n, str) for n in BSMfilepathlist):
                #self.BSMFilePathList = BSMfilepathlist
                #self.BSMNumFiles = len(self.BSMFilePathList)
                self.BSMDataFiles = []
                for path in BSMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if( (temp.Parameters == self.Parameters) and (temp.Values != 0.) ):
                        self.BSMDataFiles.append(temp)
                    else: 
                        print('File not valid: ' + path)
                        print('Parameters = ' + str(temp.Parameters) + ' and Values = ' + str(temp.Values.tolist()))
                        print('should be = ' + str(self.Parameters) + ' and != ' + str(0.))
                        raise ValueError
                        self.BSMDataFiles.append(None) 
            else:
                print('BSMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('BSMfilepathlist input should be a list !')
            raise FileNotFoundError
                  
###### Chop the BSM data sets (stored in BSMNDList, BSMDataList, BSMWeightsList, BSMParValList, BSMTargetList)
        if type(BSMNLimits) == int:
            BSMNLimits = [BSMNLimits for data in self.BSMDataFiles]
        elif type(BSMNLimits) == list and all(isinstance(n, int) for n in BSMNLimits):
            if len(BSMNLimits) != len(self.BSMDataFiles):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.BSMDataFiles)))
                raise ValueError
            elif sum([self.BSMDataFiles[i].ND >= BSMNLimits[i] for i in range(len(BSMNLimits))]
                    ) != len(self.BSMDataFiles):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: "+str([file.ND for file in self.BSMDataFiles ]))
                raise ValueError
        else:
            BSMNLimits =[file.ND for file in self.BSMDataFiles]   
            
        self.BSMNDList = BSMNLimits
        #self.BSMNData = sum(self.BSMNDataList)
        self.BSMDataList = [DF.Data[:N] for (DF, N) in zip(
            self.BSMDataFiles, self.BSMNDList)]
        self.BSMWeightsList = [DF.Weights[:N] for (DF, N) in zip(
            self.BSMDataFiles, self.BSMNDList)] 
        self.BSMValuesList = [torch.ones(N)*DF.Values for (DF, N) in zip(
            self.BSMDataFiles, self.BSMNDList)]
        
        self.BSMParValList =  [torch.ones(N, )*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.BSMNDList)]
        
        self.BSMTargetList = [torch.ones(N, dtype=torch.double) for N in self.BSMNDList] 
        
####### Load SM data (stored in SMDataFiles)
        if type(SMfilepathlist) == list:
            if all(isinstance(n, str) for n in SMfilepathlist):
                #self.SMFilePathList = SMfilepathlist
                #self.SMNumFiles = len(self.SMFilePathList)
                self.SMDataFiles = []
                for path in SMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if( (temp.Parameters == self.Parameters) and (temp.Values == 0.) ):
                       self.SMDataFiles.append(temp)
                    else:
                       print('File not valid: ' + path)
                       print('Parameters = ' + str(temp.Parameters) + ' and Values = ' + str(temp.Values.tolist()))
                       print('should be = ' + str(self.Parameters) + ' and = ' + str(0.))
                       self.SMDataFiles.append(None)                    
            else:
                print('SMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('SMfilepathlist input should be a list !')
            raise FileNotFoundError
            
####### Chop the SM data sets and join them in one (stored in SMND, SMData and SMWeights)
        if type(SMNLimits) == int:
            SMNLimits = [SMNLimits for data in self.SMDataFiles]
        elif type(SMNLimits) == list and all(isinstance(n, int) for n in SMNLimits):
            if len(SMNLimits) != len(self.SMDataFiles):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.SMDataFiles)))
                raise ValueError
            elif sum([self.SMDataFiles[i].ND >= SMNLimits[i] for i in range(len(SMNLimits))]
                    ) != len(self.SMDataFiles):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: " + str([file.ND for file in self.SMDataFiles]))
                raise ValueError
        else:
            SMNLimits = [file.ND for file in self.SMDataFiles]
        self.SMND = sum(SMNLimits)
        self.SMData = torch.cat(
            [DF.Data[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
            , 0) 
        self.SMWeights = torch.cat(
            [DF.Weights[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
            , 0)

####### Break SM data in blocks to be paired with BSM data (stored in UsedSMNDList, UsedSMDataList, UsedSMWeightsList, UsedSMParValList, UsedSMTargetList)
        BSMNRatioDataList = [torch.tensor(1., dtype=torch.double)*n/sum(self.BSMNDList
                                                                       ) for n in self.BSMNDList]
        self.UsedSMNDList = [int(self.SMND*BSMNRatioData) for BSMNRatioData in BSMNRatioDataList] 
        #self.UsedSMNData = sum(self.UsedSMNDataList)
        #self.UsedSMData = self.SMData[: self.UsedSMND]
        self.UsedSMDataList =  self.SMData[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
        
    ##### Reweighting is performed such that the SUM of the SM weights in each block equals the number of BSM data times the AVERAGE 
    ##### of the original weights. This equals the SM cross-section as obtained in the specific sample at hand, times NBSM
        self.UsedSMWeightsList = self.SMWeights[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
        self.UsedSMWeightsList = [ self.UsedSMWeightsList[i]*self.BSMNDList[i]/self.UsedSMNDList[i] for i in range(len(BSMNRatioDataList))]
        
        #ReWeighting = torch.cat([torch.ones(self.UsedSMNDList[i], dtype=torch.double).mul(self.BSMNDList[i]
                                          #  ).div(self.UsedSMNDList[i]) for i in range(len(BSMNRatioDataList))])
        #self.UsedSMWeightsList = self.SMWeights[:sum(self.UsedSMNDList)].mul(ReWeighting).split(self.UsedSMNDList)
        
        self.UsedSMParValList =  [torch.ones(N, dtype=torch.double)*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.UsedSMNDList)]
        
        self.UsedSMTargetList = [torch.zeros(N, dtype=torch.double) for N in self.UsedSMNDList]

####### Join SM with BSM data
        self.Data = torch.cat(
            [torch.cat([self.UsedSMDataList[i], self.BSMDataList[i]]
                                  ) for i in range(len(self.BSMDataList))]
            )
        self.Weights = torch.cat(
            [torch.cat([self.UsedSMWeightsList[i], self.BSMWeightsList[i]]
                                  ) for i in range(len(self.BSMWeightsList))]
            )
        self.Labels = torch.cat(
            [torch.cat([self.UsedSMTargetList[i], self.BSMTargetList[i]]
                                  ) for i in range(len(self.BSMTargetList))]
            )
        self.ParVal = torch.cat(
            [torch.cat([self.UsedSMParValList[i], self.BSMParValList[i]]
                                  ) for i in range(len(self.BSMParValList))]
            )
        
####### Return Tranining Data
    def ReturnData(self):
        return [self.Data, self.Labels, self.Weights, self.ParVal]
            
    def Report(self):
        #from tabulate import tabulate
        print('\nLoaded SM Files:')
        print(tabulate({str(self.Parameters): [ file.Values for file in self.SMDataFiles ], 
                        "#Data":[ file.ND for file in self.SMDataFiles ], 
                        "XS[pb](avg.w)":[ file.XS for file in self.SMDataFiles ]}, headers="keys"))
        print('\nLoaded BSM Files:')
        print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], 
                        "#Data":[ file.ND for file in self.BSMDataFiles ], 
                        "XS[pb](avg.w)":[ file.XS for file in self.BSMDataFiles ]}, headers="keys"))
        print('\nPaired BSM/SM Datasets:\n')
        print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], "#Ev.BSM": self.BSMNDList
                        , "#Ev.SM": self.UsedSMNDList,
                        "sum.w SM\/XSSM": [(self.UsedSMWeightsList[i].sum())/(self.SMWeights.mean()) for i in range(len(self.BSMDataFiles))]
                       }, headers="keys"))

TD = OurTrainingData(['/data3/Training/TrainingData/GWData/ChPsm.h5', '/data3/MadGraph/testSM1.h5'], 
                     ['/data3/Training/TrainingData/GWData/ChPgw2.h5', '/data3/Training/TrainingData/GWData/ChPgwm2.h5',
                      '/data3/Training/TrainingData/GWData/ChPgw5.h5'],
                     ['GW[TeV**-2]'], verbose=False, SMNLimits = [ 10, 5 ], BSMNLimits = [ 2, 3, 4 ] )
#print(TD.SMData.size())
#print(TD.SMData.dtype)
#print([w.size() for w in TD.BSMWeightsList])
#print([w.dtype for w in TD.BSMWeightsList])
TD.Report()
print(TD.Data.size())
print(TD.Weights.size())
print(TD.Labels.size())
print(TD.ParVal.size())

print(TD.Labels)

print(TD.ParVal)

print(TD.Weights)

print('%.8f' %TD.Weights[3])

print('%.8f' %TD.BSMDataFiles[0].Weights[0])

print('%.8f' %TD.Weights[0])

print('%.8f' %(TD.SMDataFiles[0].Weights[0]*2./3.))


In [ ]:
class OurTrainingData():
    def __init__(self, SMfilepathlist, BSMfilepathlist, parameters, SMNLimits="NA", BSMNLimits="NA", verbose=True): 
        self.Parameters = parameters
        print('Loading Data Files with Parameters: ' + str(parameters) ) 
        if len(self.Parameters)!= 1: print('Only 1D Implemented in Training !')        
####### Load SM data
        if type(SMfilepathlist) == list:
            if all(isinstance(n, str) for n in SMfilepathlist):
                self.SMFilePathList = SMfilepathlist
                self.SMNumFiles = len(self.SMFilePathList)
                def ReadSMFile(path): 
                    print('\nReading SM File ...' + path)
                    file = h5py.File(path, 'r')
                    if list(file.keys()) == ['Data', 'Info', 'Parameters', 'Values', 'Weights']:
                        if file['Parameters'][()] == self.Parameters:
                            if file['Values'][()] == [0] * len(file['Values'][()]):
                                if verbose: print('##### File Info:\n' +
                                    file['Info'][()][0] + '\n#####')
                                return [file['Info'][()][0], torch.Tensor(file['Values'][()]), torch.Tensor(file['Data'][()]), 
                                    torch.Tensor(file['Weights'][()])]    
                            else:
                                print('File: ' + path + ' is of BSM type!')
                                return None
                        else:
                            print('Parameters key is ' + str(file['Parameters'][()]) + '. It should be '+ str(self.Parameters))
                            return None
                    else:
                        print('File format not valid: ' + path)
                        return None
                ImportedFiles = list(map(ReadSMFile, self.SMFilePathList))
                if not None in ImportedFiles:
                    self.SMInfoList, self.SMValuesList, self.SMFilesDataList, \
                        self.SMFilesWeightsList = list(map(list, zip(*ImportedFiles)))
                    self.SMFilesNDataList =  list(map(len, self.SMFilesDataList))
                    self.SMSigmaList = list(map(np.average, self.SMFilesWeightsList))                    
            else:
                print('SMfilepathlist input should be a list of strings !')
        else:
            print('SMfilepathlist input should be a list !')

####### Join SM data and SM weigths
        if type(SMNLimits) == int:
            SMNLimits = [SMNLimits for data in self.SMFilesDataList]
        elif type(SMNLimits) == list and all(isinstance(n, int) for n in SMNLimits):
            if len(SMNLimits) != len(self.SMFilesDataList):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.SMFilesDataList)))
            elif sum([self.SMFilesNDataList[i] >= SMNLimits[i] for i in range(len(SMNLimits))]
                    ) != len(self.SMFilesNDataList):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: "+str(self.SMFilesNDataList))
        else:
            SMNLimits = [len(data) for data in self.SMFilesDataList]
            
        self.SMNDataList = SMNLimits
        self.SMDataList = [self.SMFilesDataList[i][:self.SMNDataList[i]] \
                           for i in range(self.SMNumFiles)]
        self.SMWeightsList = [self.SMFilesWeightsList[i][:self.SMNDataList[i]] \
                           for i in range(self.SMNumFiles)]
####### Join SM Data
        self.SMData = torch.cat(self.SMDataList, 0)  
        self.SMWeights = torch.cat(self.SMWeightsList, 0)
        self.SMNData = sum(self.SMNDataList)
####### Load BSM data
        if type(BSMfilepathlist) == list:
            if all(isinstance(n, str) for n in BSMfilepathlist):
                self.BSMFilePathList = BSMfilepathlist
                self.BSMNumFiles = len(self.BSMFilePathList)
                def ReadBSMFile(path): 
                    print('\nReading BSM File ...' + path)
                    file = h5py.File(path, 'r')
                    if list(file.keys()) == ['Data', 'Info', 'Parameters', 'Values', 'Weights']:
                        if file['Parameters'][()] == self.Parameters:
                            if file['Values'][()] != [0] * len(file['Values'][()]):
                                if verbose: print('##### File Info:\n' +
                                    file['Info'][()][0] + '\n#####')
                                return [file['Info'][()][0], torch.Tensor(file['Values'][()]), torch.Tensor(file['Data'][()]), 
                                    torch.Tensor(file['Weights'][()])]    
                            else:
                                print('File: ' + path + ' is of SM type!')
                                return None
                        else:
                            print('Parameters key is ' + str(file['Parameters'][()]) + '. It should be '+ str(self.Parameters))
                            return None
                    else:
                        print('File format not valid: ' + path)
                        return None
                ImportedFiles = list(map(ReadBSMFile, self.BSMFilePathList))
                if not None in ImportedFiles:
                    self.BSMInfoList, self.BSMValuesList, self.BSMDataList, \
                        self.BSMWeightsList = list(map(list, zip(*ImportedFiles)))
                    self.BSMNDataList = list(map(len, self.BSMDataList))
                    self.BSMSigmaList = list(map(np.average, self.BSMWeightsList))
            else:
                print('BSMfilepathlist input should be a list of strings !')
        else:
            print('BSMfileathlist input should be a list !')
            
        if type(BSMNLimits) == int:
            BSMNLimits = [BSMNLimits for data in self.BSMDataList]
        elif type(BSMNLimits) == list and all(isinstance(n, int) for n in BSMNLimits):
            if len(BSMNLimits) != len(self.BSMDataList):
                print("--> Please input %d integers to chop each SM file."%(
                    len(self.BSMDataList)))
            elif sum([self.BSMNDataList[i] >= BSMNLimits[i] for i in range(len(BSMNLimits))]
                    ) != len(self.BSMNDataList):
                print("--> Some chop limit larger than available data in the corresponding file.")
                print("--> Lengths of the files: "+str(self.BSMNDataList))
            else:
                self.BSMNDataList = BSMNLimits
                self.BSMDataList = [self.BSMDataList[i][:self.BSMNDataList[i]] \
                                   for i in range(len(self.BSMDataList))]
####### Break SM data in blocks to be paired with BSM data 
        BSMNRatioDataList = list(map(lambda n: n/sum(self.BSMNDataList), self.BSMNDataList))
        self.SMNSampleList = [int(self.SMNData*BSMNRatioData) for BSMNRatioData in BSMNRatioDataList] 
        #self.SMValues = torch.cat([torch.ones(self.SMNSampleList[i])*self.BSMValuesList[i]
        #                     for i in range(len(BSMNRatioDataList))])
        self.SMNSample = sum(self.SMNSampleList)
        self.SMSample = self.SMData[: self.SMNSample]
        ReWeighting = torch.cat([torch.ones(self.SMNSampleList[i])*self.BSMNDataList[i]/self.SMNSampleList[i] 
                                 for i in range(len(BSMNRatioDataList))])
        #print(ReWeighting)
        #print(sum(self.SMWeights))
        self.SMWeights = self.SMWeights[:self.SMNSample].mul(ReWeighting)
        print((self.SMWeights.sum())/np.average(self.SMWeights))
        print((self.SMWeights.sum())/self.SMWeights.mean())
        print(len(self.SMWeights))
        #print('Number of points in SM samples: '+str(self.SMNSampleList))
        
        #print('Number of points in BSM samples: '+str(self.BSMNDataList))
####### Create training labels
        self.SMLabels = torch.zeros(self.SMNSample).split(self.SMNSampleList)
        self.BSMLabels = torch.ones(sum(self.BSMNDataList)).split(self.BSMNDataList)
####### Create training parameter values
        self.ParameterValuesList = [torch.ones(self.SMNSampleList[i] + self.BSMNDataList[i])*\
                                    (self.BSMValuesList[i]) for i in range(len(self.BSMDataList))]
####### Join SM with BSM data
        self.SMSampleList = torch.split(self.SMSample, self.SMNSampleList, 0)
        self.SMWeightsList = torch.split(self.SMWeights, self.SMNSampleList, 0)
        self.DataList = [torch.cat([self.SMSampleList[i], self.BSMDataList[i]]
                                  ) for i in range(len(self.BSMDataList))]
        self.LabelList = [torch.cat([self.SMLabels[i], self.BSMLabels[i]]
                                  ) for i in range(len(self.BSMDataList))]
        self.WeightsList = [torch.cat([self.SMWeightsList[i], self.BSMWeightsList[i]]
                                  ) for i in range(len(self.BSMNDataList))]
        self.Data = torch.cat(self.DataList)
        self.Labels = torch.cat(self.LabelList)
        self.Weights = torch.cat(self.WeightsList)
        self.TrainingParameters = torch.cat(self.ParameterValuesList)
####### Output Tranining Data
    def ReturnData(self):
        return [self.Data, self.Labels, self.Weights, self.TrainingParameters]

    def Report(self):
        #from tabulate import tabulate
        print('\nLoaded Files:\n')
        print(tabulate({str(self.Parameters): self.SMValuesList, 
                        "#Events": self.SMFilesNDataList, "XS[pb](avg.w)": self.SMSigmaList}, headers="keys"))
        print(' ')
        print(tabulate({ str(self.Parameters): self.BSMValuesList, 
                        "#Events": self.BSMNDataList, "XS[pb](avg.w)": self.BSMSigmaList}, headers="keys"))
        print('\nPaired BSM/SM Datasets:\n')
        print(tabulate({str(self.Parameters): self.BSMValuesList, "#Ev.BSM": self.BSMNDataList
                        , "#Ev.SM": self.SMNSampleList, 
                        "sum.w BSM\/XSBSM": [sum(self.BSMWeightsList[i])/self.BSMSigmaList[i] for i in range(len(self.BSMWeightsList))],
                        "sum.w SM\/XSSM": [sum(self.SMWeightsList[i])//np.average(self.SMSigmaList) for i in range(len(self.SMWeightsList))]
                       }, headers="keys"))

In [ ]:
 t = OurTrainingData(['/data3/Training/TrainingData/GWData/ChMsm.h5'],\
                    ['/data3/Training/TrainingData/GWData/ChMgw10.h5'], ['GW[TeV**-2]'],
                    SMNLimits=[19000])
t.Report()

In [ ]:
t = OurTrainingData(['/data3/MadGraph/testSM1.h5','/data3/MadGraph/testSM2.h5'],\
                    ['/data3/MadGraph/testBSM1.h5', '/data3/MadGraph/testBSM2.h5'], ['GW'],
                   SMNLimits=[5000, 2000], BSMNLimits=[10000, 5000])
t.Report()
#o = t.ReturnData()
#print(o)

In [ ]:
t.SMDataList[0][:np.infty]

In [ ]:
arr1 = np.array([1, 2, 3])

arr2 = np.array([4, 5, 6])

arr = np.concatenate((arr1, arr2))

print(arr)

In [ ]:
a = [1, 2, 4, 5]
b = [0, 1, 3, 5]
sum([a[i] > b[i] for i in range(len(a))])

In [ ]:
class OurTrainingData():
    def __init__(self, filepathlist, parameters = ['GW']): 
        self.Parameters = parameters
        print('Only 1D Implemented in Training !') if len(self.Parameters)!=1 else None
        if type(filepathlist) == list:
            if all(isinstance(n, str) for n in filepathlist):
                self.FilePathList = filepathlist
                #print('Reading Files ...')
                #print(*self.FilePathList, sep = '\n')
                def ReadFile(path): 
                    print('Reading File ...' + path)
                    file = h5py.File(path, 'r')
                    #print(list(file.keys()))
                    if list(file.keys()) != ['Data', 'Info', 'Parameters', 'Values', 'Weights']:
                        print('File format not valid: ' + path)
                        return None
                    else:
                        return [file['Info'][()][0], np.array(file['Parameters'][()]), np.array(file['Values'][()]), np.array(file['Data'][()]), 
                                np.array(file['Weights'][()])]
                ImportedFiles = list(map(ReadFile, self.FilePathList))
                if not None in ImportedFiles:
                    self.InfoList, self.ParametersList, self.ValuesList, self.DataList, self.WeigthsList = list(map(list, zip(*ImportedFiles)))
                    self.NDataList =  list(map(len, self.DataList))
                    def f(x) : 
                        if x.sum() == 0:
                            return 0
                        else:
                            return 1
                    self.TargetList = list(map(f, self.ValuesList))
                    if np.all(self.ParametersList !=  self.Parameters): print('Not all files have ' + str(self.Parameters) + 'Parameters !')
            else:
                print('Input should be a list of strings !')
        else:
            print('Input should be a list !')
    def ReturnData(self):
        output = []
        for i in range (0, len(self.InfoList)):
            print('here' + str(self.TargetList[i]))
            targ = np.empty(self.NDataList[i])
            targ.fill(self.TargetList[i])
            output.extend(targ)
        return np.array(output)
 #       def f(target,):
  #          target = x[0]
   #         Data = x[1]
    #        tl = np.empty(len(Data))
     #       tl.fill(target)
                
    def Report(self):
        from tabulate import tabulate
        print('Report:')
        print(tabulate({"File": self.FilePathList, "Info": self.InfoList, "Parameters": self.ParametersList, "Values": self.ValuesList, 
                        "Target": self.TargetList, "#Events": self.NDataList}, headers="keys"))
        #print(*self.FilePathList, sep = '\n')
        #print(list(map(len, self.DataList)))
        #print(list(self.ParametersList))
        #print(list(self.ValuesList))       
#def Import_pData(datafilepath):
#    print(datafilepath)
#    datafile = h5py.File(datafilepath, 'r')
#    print(list(datafile.keys()))
#    if list(datafile.keys()) != ['Data', 'Weights']:
#        print('File format not vadid: ' + datafilepath)
#        return 1
#    else:
#        data=datafile['Data']
#        weights=datafile['Weights']
#        #print(list(weights))
#        return np.array([weights,data])

In [ ]:
a = np.array([0,0.])
b = np.array(['f','g'])
a.sum()
print(a[a.nonzero()])
print(b[a.nonzero()])
print('here') if True else None
a = np.array([[1,2,3],[3,4,5]])
len(a)
a = []
a.append([1])
print(a)

In [ ]:
def f(x): return -x, x
a, b, = list(map(f,[3,4]))
print(a)
list(map(list, zip(*[[1,2],[3,4]])))

In [ ]:
torch.Tensor([[1,2,3],[2,4,5]])

In [ ]:
a = [3, 2]
a.sort()
print(a)

In [ ]:
data[0:2]

In [ ]:
weights[0:3]

In [ ]:
class QuadraticNet(nn.Module):
    def __init__(self):
        super(QuadraticNet, self).__init__()
        self.fclist1  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc1     = nn.Linear(n_neurons[-1], 1)
        
        self.fclist2  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc2     = nn.Linear(n_neurons[-1], 1)
        
    def forward(self, x):
        x, p    = x[:, :input_dim], x[:, input_dim:-1].squeeze() 
        x1 = x2 = x
        
        for i, l in enumerate(self.fclist1):
            x1 = torch.sigmoid(l(x1))
        x1     = self.fc1(x1).squeeze()
        
        for i, l in enumerate(self.fclist2):
            x2 = torch.sigmoid(l(x2))
        x2     = self.fc2(x2).squeeze()
        
        capf   = torch.log(
            ((1 + torch.mul(x1, p))**2 + (torch.mul(x2, p))**2))
        
        return torch.sigmoid(capf).view(-1, 1)
    
    def get_L1max(self, value_per_unit):
        L1_max = []
        for m in model_plus.children():
            if isinstance(m, nn.Linear):
                L1_max.append(m.weight.size(0)*m.weight.size(1) \
                                  * value_per_unit)
            else:
                for mm in m:
                    L1_max.append(mm.weight.size(0)*mm.weight.size(1)\
                                      *value_per_unit)
        self.L1max_list = L1_max
    
    def clip_L1(self):
        counter = 0
        for m in model_plus.children():
            if isinstance(m, nn.Linear):
                counter += 1
                with torch.no_grad():
                    designated_L1 = self.L1max_list[counter-1]
                    if designated_L1 == value_per_unit*n_neurons[0]*n_neurons[1]:
                        continue                                #skipping first layer
                    L1 = m.weight.abs().sum()
                    m.weight.masked_scatter_(L1>self.L1max_list[counter-1],
                                            m.weight/L1*self.L1max_list[counter-1])
            else:
                for mm in m:
                    counter +=1
                    with torch.no_grad():
                        designated_L1 = self.L1max_list[counter-1]
                        if designated_L1 == value_per_unit*n_neurons[0]*n_neurons[1]:
                            continue                            #skipping first layer
                        L1 = mm.weight.abs().sum()
                        mm.weight.masked_scatter_(L1>designated_L1,
                                            mm.weight/L1*designated_L1)
    
    def calculate_ratio(self, points):
        with torch.no_grad():
            y = self(points)
        return y/(1-y)

In [ ]:
l=np.array([[1,2],[4,5]]).transpose()
print(l)

In [ ]:
l[0][0]

In [ ]:
f=2
print(f)

In [ ]:
del test

In [ ]:
test3 = h5py.File('/data3/MadGraph/test.h5', 'r')
list(test3.keys())

In [ ]:
test = h5py.File('/data3/MadGraph/test.h5', 'r')
print(list(test.keys()))
data=test['Dataset1']
print(list(data))
data[2]

# Multiple Functions

In [ ]:
#### helper functions ####

def convert_angles_in_data(data, angle_pos):
    nonangle_pos = list(set(range(data.shape[1]))-set(angle_pos))
    nonangle_pos.sort()

    catdata = torch.cat((data[:, angle_pos].cos_(), 
                         data[:, angle_pos].sin_(),
                         data[:, nonangle_pos]), 1)
    
    return catdata

def append_constant(data, constant):
    return torch.cat((data, torch.ones(data.size(0), 1)*float(constant)), 1)

def report_ETA(beginning, start, epochs, e, loss):
    time_elapsed = time.time() - start
    time_left    = str(datetime.timedelta(
        seconds=((time.time() - beginning)/(e+1)*(epochs-(e+1)))))
    print('Training epoch %s (took %.2f sec, time left %s sec) loss %.8f'%(
        e, time_elapsed, time_left, loss))
    return time.time()

def simpleplot(tsm, tbsm, title, sep, p, deltap):
    plt.figure(figsize=(8, 6))
    ax = plt.subplot()
    plt.hist(tsm,  50, alpha=0.5, label='SM')
    plt.hist(tbsm, 50, alpha=0.5, label='BSM')
    plt.title(title)
    plt.legend(loc='upper right')
    
    plt.text(x=0.05, y=0.85, transform=ax.transAxes, 
         s='sep = %.3f\np = %.3f +/- %.3f'%(sep, p, deltap), 
         bbox=dict(facecolor='blue', alpha=0.2))
    
    plt.savefig(outputfolder + '/' + title+'.pdf')
    plt.show()

def combine_pm(tsm_plus, tbsm_plus, tsm_minus, tbsm_minus, e):
    len_sm    = min(len(tsm_plus), len(tsm_minus))
    len_bsm   = min(len(tbsm_plus), len(tbsm_minus))
    
    tsm       = (tsm_plus[:len_sm] + tsm_minus[:len_sm])
    tbsm      = (tbsm_plus[:len_bsm] + tbsm_minus[:len_bsm])
    
    mu_sm     = tsm.mean().item()
    mu_bsm    = tbsm.mean().item()
    sigma_sm  = tsm.std().item()
    sigma_bsm = tbsm.std().item()
    med_sm    = tsm.median().item()
    
    sep    = (mu_sm - mu_bsm)/sigma_bsm
    p      = 1.*len([i for i in tbsm if i > med_sm])/len(tsm)
    
    delta1 = (p * (1 - p)/min(len_sm, len_bsm))**0.5
    delta2 = (sigma_sm/sigma_bsm) * np.exp(-((mu_bsm - mu_sm)**2)/(
            2 * sigma_bsm**2))/(2*(n_meas**0.5))
    deltap = (delta1**2 + delta2**2)**0.5
    
    title = '%s, %s%s, combined, %s, N=%d, epochs=%d'%(
                     outputheader, str(n_neurons), bsm_op, bsm_test, N, e)

    simpleplot(tsm, tbsm, title, sep, p, deltap)
    
    return (p, deltap)

def conclude(title, p_history, outputfolder):
    title = 'Test p value history - ' + title
    plt.subplots(figsize = (8,4))
    plt.xscale('log')
    plt.ylim(top = max(map(lambda entry: entry[1], p_history[1:])))
    plt.title(title)
    plt.xlabel('epochs')
    plt.ylabel('p value')

    x    = list(map(lambda l: l[0], p_history))
    y    = list(map(lambda l: l[1], p_history))
    yerr = list(map(lambda l: l[2], p_history))
    plt.errorbar(x, y, yerr = yerr)
    
    plt.hlines(y=0.05, xmin=x[0], xmax=x[-1], colors='red')

    plt.savefig(outputfolder + title + '.pdf')
    plt.show()
    plt.close()
    np.savetxt(outputfolder + title + '.csv', p_history)

In [ ]:
def multiply_by_constant(data, constant, pos, inplace=True):
    updatevalue = torch.mul(data[:, pos], constant).view(-1, 1)
    if inplace:
        return torch.cat((data[:, :pos], updatevalue, data[:, pos+1:]), 1)
    else:
        return torch.cat((data[:, :pos], data[:, pos+1:], updatevalue), 1)

def take_ratio(data, num_pos, den_pos):
    ratio = data[:, num_pos]/data[:, den_pos]
    data[:, num_pos] = ratio
    
    return data

# Network and Loss

In [ ]:
class QuadraticNet(n_neurons):
    def __init__(self):
        super(QuadraticNet, self).__init__()
        self.fclist1  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc1     = nn.Linear(n_neurons[-1], 1)
        
        self.fclist2  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc2     = nn.Linear(n_neurons[-1], 1)
        
    def forward(self, x):
        x, p    = x[:, :input_dim], x[:, input_dim:-1].squeeze() 
        x1 = x2 = x
        
        for i, l in enumerate(self.fclist1):
            x1 = torch.sigmoid(l(x1))
        x1     = self.fc1(x1).squeeze()
        
        for i, l in enumerate(self.fclist2):
            x2 = torch.sigmoid(l(x2))
        x2     = self.fc2(x2).squeeze()
        
        capf   = torch.log(
            ((1 + torch.mul(x1, p))**2 + (torch.mul(x2, p))**2))
        
        return torch.sigmoid(capf).view(-1, 1)
    
    def get_L1max(self, value_per_unit):
        L1_max = []
        for m in model_plus.children():
            if isinstance(m, nn.Linear):
                L1_max.append(m.weight.size(0)*m.weight.size(1) \
                                  * value_per_unit)
            else:
                for mm in m:
                    L1_max.append(mm.weight.size(0)*mm.weight.size(1)\
                                      *value_per_unit)
        self.L1max_list = L1_max
    
    def clip_L1(self):
        counter = 0
        for m in model_plus.children():
            if isinstance(m, nn.Linear):
                counter += 1
                with torch.no_grad():
                    designated_L1 = self.L1max_list[counter-1]
                    if designated_L1 == value_per_unit*n_neurons[0]*n_neurons[1]:
                        continue                                #skipping first layer
                    L1 = m.weight.abs().sum()
                    m.weight.masked_scatter_(L1>self.L1max_list[counter-1],
                                            m.weight/L1*self.L1max_list[counter-1])
            else:
                for mm in m:
                    counter +=1
                    with torch.no_grad():
                        designated_L1 = self.L1max_list[counter-1]
                        if designated_L1 == value_per_unit*n_neurons[0]*n_neurons[1]:
                            continue                            #skipping first layer
                        L1 = mm.weight.abs().sum()
                        mm.weight.masked_scatter_(L1>designated_L1,
                                            mm.weight/L1*designated_L1)
    
    def calculate_ratio(self, points):
        with torch.no_grad():
            y = self(points)
        return y/(1-y)

In [ ]:
QuadraticNet()

In [ ]:
class QuadraticNetReLU(QuadraticNet):
    def __init__(self):
        super(QuadraticNetReLU, self).__init__()
        self.fclist1  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc1     = nn.Linear(n_neurons[-1], 1)
        
        self.fclist2  = nn.ModuleList([nn.Linear(n_neurons[i], 
            n_neurons[i+1]) for i in range(len(n_neurons)-1)])
        self.fc2     = nn.Linear(n_neurons[-1], 1)
        
    def forward(self, x):
        x, p    = x[:, :input_dim], x[:, input_dim:-1].squeeze()
        x1 = x2 = x
        
        for i, l in enumerate(self.fclist1):
            x1 = torch.nn.functional.relu(l(x1))
        x1     = self.fc1(x1).squeeze()
        
        for i, l in enumerate(self.fclist2):
            x2 = torch.nn.functional.relu(l(x2))
        x2     = self.fc2(x2).squeeze()
        
        capf   = torch.log(
            ((1 + torch.mul(x1, p))**2 + (torch.mul(x2, p))**2))
        
        return torch.sigmoid(capf).view(-1, 1)

In [ ]:
class _Loss(Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction

class WeightedMSELoss(_Loss):
    __constants__ = ['reduction']

    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(WeightedMSELoss, self).__init__(size_average, reduce, reduction)

    def forward(self, input, target, weight):
        return torch.mean(torch.mul(weight, (input - target)**2))

### Read Data - Train Data

### Training 

In [ ]:
def train(model, X_train, y_train):
    model.get_L1max(value_per_unit)
    optimiser           = torch.optim.Adam(model.parameters(), lr)
    criterion           = WeightedMSELoss()
    
    if use_gpu:
        model                 = model.cuda()
        X_train, y_train      = X_train.cuda(), y_train.cuda()

    print(" =================== BEGINNING TRAIN ==================== ")
    beginning = start = time.time()

    for e in range(epochs):
        output          = model(X_train)
        loss            = criterion(output, y_train, X_train[:, -1:])

        if (e+1) % verbose_prd  == 0:
            start       = report_ETA(beginning, start, epochs, e+1, loss)
            if save_history:
                generictitle = '%s, %s, %s, N=%d, epochs=%d'%(outputheader, pm, str(n_neurons), N, e+1)
                torch.save({'state_dict': model.state_dict()}, outputfolder + 
                                generictitle + '.pth')
                modelparams = [w.detach().tolist() for w in model.parameters()]
                np.savetxt(outputfolder + generictitle + '.csv', modelparams, '%s')            
            
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        model.clip_L1()
        
    print(" ===================   END OF TRAIN   =================== ")
    
    return model

## Adding ratio of PT/pt instead of PT

In [ ]:
def read_data(pm):
    sm_file        = sm_filename_fn(pm)    
    train_size     = len(bsm_coef)*N
    
    smdata         = h5py.File(datafolder + sm_filename + '_' + pm + '_nlo.h5', 'r') 
    nsm            = smdata['Number'][()]
    
    if isinstance(nsm, np.ndarray):
        nsm = nsm[0]
    
    smdata         = torch.Tensor(smdata['Data'][()])
    smdata         = take_ratio(smdata, -2, -3)
    
    smdata_lst     = [append_constant(smdata[i*N:(i+1)*N, :], bsm_coef[i]
                                 ) for i in range(len(bsm_coef))]    # wilson coefficient G for SM
    smdata_lst.append(
        append_constant(smdata[train_size:, :], bsm_test))           # wilson coefficient G for SM (test)
    smdata         = torch.cat(smdata_lst, 0)
    
    #################################################################################################
    #### be careful! as this changes the position of the column of weight and wilson coefficient ####
    nlo_w_mean_sm  = smdata[:, -2].mean()
    smdata         = multiply_by_constant(smdata, 
                          1./nlo_w_mean_sm, -2, inplace=False)     # sigma_g/sigma_0 = 1 for SM, NLO
    #################################################################################################
    
    smdata         = append_constant(smdata, 0)                      # training target y = 0 for SM
    
    nbsm_dic       = {c: (h5py.File(datafolder + bsm_filename_fn(bsm_op, c, pm), 
                                    'r')['Number'][()]) for c in bsm_coef}
        
    if isinstance(nbsm_dic[bsm_coef[0]], np.ndarray):
        nbsm_dic = {c: torch.as_tensor(nbsm_dic[c][0]) for c in bsm_coef}
    else:
        nbsm_dic = {c: torch.as_tensor(nbsm_dic[c]) for c in bsm_coef}
    
    
    #nbsm_dic       = {c: (torch.as_tensor(h5py.File(datafolder + bsm_filename(bsm_op, c)\
    #                + '_' + pm + '_nlo.h5', 'r')['Number'][()])) for c in bsm_coef}
    
    bsmdata_lst    = [torch.Tensor(h5py.File(datafolder + bsm_filename_fn(bsm_op, c, pm),
                                             'r')['Data'][()])[:N, :] for c in bsm_coef]
    
    print('============ nsm: %s ============'%(str(nsm)))
    for i in range(len(bsm_coef)):
        print('========== nbsm %i: %s =========='%(i, str(nbsm_dic[bsm_coef[i]])))
    
    bsmdata_lst    = [take_ratio(bsmdata_lst[i], -2, -3) for i in range(len(bsmdata_lst))]

    bsmdata_lst    = [append_constant(bsmdata_lst[i], bsm_coef[i]
                                 ) for i in range(len(bsmdata_lst))] # wilson coefficient G for BSM
    
    nlo_w_mean_bsm = [bsmdata_lst[i][:, -2].mean() for i in range(len(bsmdata_lst))]   
    
    bsmdata_lst    = [multiply_by_constant(bsmdata_lst[i], (nbsm_dic[bsm_coef[i]]/nsm)/nlo_w_mean_bsm[i],
                            -2, inplace=False) for i in range(len(bsmdata_lst))] # sigma_g/sigma_0 = nbsm/nsm for BSM, NLO
    
    bsmdata        = torch.cat(bsmdata_lst, 0)
    bsmdata        = append_constant(bsmdata, 1)                     # training target y = 1 for BSM

    traindata      = torch.cat((smdata[:train_size, :], bsmdata), 0)
    traindata      = traindata[torch.randperm(traindata.size(0))]

    if convert_ang:
        traindata  = convert_angles_in_data(traindata, angle_pos)

    X_train        = traindata[:, :-1] 
    y_train        = traindata[:, -1].reshape(-1, 1)
    
    title = 'Scaling (input dim %d, ratio), %s, %s'%(input_dim, pm, bsm_coef)

    if os.path.isfile(scalingfolder + title + '.csv'):
        print('Reading scaling from: \n%s...'%(scalingfolder + title + '.csv'))
        
        scalingPars  = np.loadtxt(open(scalingfolder + title + '.csv', 'r'))
        X_train_mean = torch.from_numpy(scalingPars[:input_dim+1]).type(torch.FloatTensor)
        X_train_std  = torch.from_numpy(scalingPars[input_dim+1:]).type(torch.FloatTensor)
    else:
        X_train_mean           = X_train[:,  :input_dim+1].mean(0)
        X_train_std            = X_train[:,  :input_dim+1].std(0)
    
    # normalisation
    X_train[:, :input_dim+1] = (X_train[:, :input_dim+1] - X_train_mean)/X_train_std
    
    #################################################################################################
    
    bsmdata_test   = h5py.File(datafolder + bsm_filename(bsm_op, bsm_test) +\
                            '_' + pm +'_nlo.h5', 'r')
    nbsm_dic[bsm_test] = (bsmdata_test['Number'][()])
    
    if isinstance(nbsm_dic[bsm_test], np.ndarray):
        nbsm_dic[bsm_test] = torch.as_tensor(nbsm_dic[bsm_test][0])
    else:
        nbsm_dic[bsm_test] = torch.as_tensor(nbsm_dic[bsm_test])
    
    bsmdata_test   = torch.Tensor(bsmdata_test['Data'][()])
    bsmdata_test   = take_ratio(bsmdata_test, -2, -3)
    
    bsmdata_test   = append_constant(bsmdata_test, bsm_test)         # wilson coefficient G for BSM
    
    #################################################################################################
    #### be careful! as this changes the position of the column of weight and wilson coefficient ####
    nlo_w_mean_bsm_test = bsmdata_test[:, -2].mean() 
    
    bsmdata_test   = multiply_by_constant(bsmdata_test, 
        (nbsm_dic[bsm_test]/nsm)/nlo_w_mean_bsm_test, -2, inplace=False)# sigma_g/sigma_0 = nbsm/nsm for BSM
    #################################################################################################
    
    bsmdata_test   = append_constant(bsmdata_test, 1)                # testing target y = 1 for BSM

    if bsm_test in bsm_coef:
        bsmdata_test  = bsmdata_test[N:, :]

    smdata_test    = smdata[train_size:,  :]
    
    #testdata       = torch.cat((smdata[train_size:, :], bsmdata_test), 0)
    #testdata       = testdata[torch.randperm(testdata.size(0))]

    if convert_ang:
        smdata_test   = convert_angles_in_data(smdata_test,  angle_pos)
        bsmdata_test  = convert_angles_in_data(bsmdata_test, angle_pos)
    
    X_test_sm      = smdata_test[:,  :-1] 
    X_test_bsm     = bsmdata_test[:, :-1]
    
    # normalisation
    X_test_sm[:,  :input_dim+1]  = (X_test_sm[:, :input_dim+1] - X_train_mean)/X_train_std
    X_test_bsm[:, :input_dim+1] = (X_test_bsm[:, :input_dim+1] - X_train_mean)/X_train_std
    
    if not os.path.isfile(scalingfolder + title + '.csv'):
        print('Saving scaling parameters at: \n%s...'%(scalingfolder + title + '.csv'))
        
        np.savetxt(outputfolder + title + '.csv', torch.cat(
        [X_train_mean.reshape(-1, 1), X_train_std.reshape(-1, 1)], 0).numpy())
        
    return X_train, y_train, X_test_sm, X_test_bsm, nsm, nbsm_dic

### Fixed Parameters

In [ ]:
datafolder    = '/home/chen/Documents/DibosonProcessData_NLO_NewBias/'
scalingfolder = '/home/chen/Documents/OutputQuadratic_NLO_NewBias_alt/'
outputfolder  = '/home/chen/Documents/OutputQuadratic_NLO_NewBias_alt/'
outputheader  = 'Newbias'

sm_filename_fn   = lambda pm: 'sm_%s_nlo.h5'%(pm)
bsm_filename_fn  = lambda g, c, pm: '%s%s_%s_nlo.h5'%(g, c, pm)

### Dynamic Parameters

In [ ]:
angle_pos     = [3, 5]
convert_ang   = True

use_gpu       = True
input_dim     = 8 if not convert_ang else 10
n_neurons     = [input_dim, 32, 32, 32]
#N             = int(18e4)
N             = int(2e5)
lr            = 1e-3
epochs        = 10000
verbose_prd   = 1000
n_meas        = 4000
value_per_unit= 1.0
save_history  = True

In [ ]:
outputheader  = 'NewBias-GW-vanilla'
bsm_op        = 'GW'
bsm_coef      = ['-1e-7', '-5e-8', '-2e-8', '2e-8', '5e-8', '1e-7']
bsm_test      = '2e-8'

In [ ]:
pm                  = 'plus'
X_train, y_train, X_test_sm, X_test_bsm, nsm, nbsm_dic = read_data(pm)
model_plus          = QuadraticNetReLU()
model_plus          = train(model_plus, X_train, y_train)

pm                    = 'minus'
X_train, y_train, X_test_sm, X_test_bsm, nsm, nbsm_dic = read_data(pm)
model_minus           = QuadraticNetReLU()
model_minus           = train(model_minus, X_train, y_train)